<a href="https://colab.research.google.com/github/Dadoyen2/Geometry-Optimization/blob/main/TEST_geometry_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import numpy as np
import os
import contextlib
from io import StringIO
import math
import os
import contextlib
from io import StringIO

In [3]:
def get_coordinate(url):
    atoms = []
    response = requests.get(url)
    response.raise_for_status()
    lines = response.text.splitlines()

    # Start parsing for atom data directly after identifying the pattern
    for line in lines:
        parts = line.split()
        # Check if line contains a potential atom data pattern
        if len(parts) >= 4 and parts[3].isalpha():
            try:
                # Attempt to parse the first three values as coordinates
                x, y, z = map(float, parts[:3])
                atom_type = parts[3]
                atoms.append((atom_type, x, y, z))
            except ValueError:
                # Skip lines that don't fit the pattern (ensures generality)
                continue

    return atoms

def print_atom_coordinates(atoms):
    print(f"The input file has: {len(atoms)} atoms")
    print("Atoms and coordinates (in Å):")
    for atom in atoms:
        print(f"{atom[0]:<2} {atom[1]:>8.4f} {atom[2]:>8.4f} {atom[3]:>8.4f}")

# Example URL to use with the general extraction function
url = 'https://raw.githubusercontent.com/Dadoyen2/Geometry-Optimization/main/ethane.mol2'
atoms = get_coordinate(url)
#print_atom_coordinates(atoms)




In [4]:

# Define bond parameters (k_b in kcal/mol/Å^2, r_0 in Å)
bond_params = {
    ('C', 'H'): (350, 1.11),  # Example: k_b and r_0 for C-H bond
    ('C', 'C'): (300, 1.53),  # Example: k_b and r_0 for C-C bond
}

# Define bond cutoffs (min and max cutoff for bond length detection)
bond_cutoffs = {
    ('C', 'H'): (0.5, 1.5),  # Minimum and maximum cutoff for C-H bond detection
    ('C', 'C'): (1.0, 2.0),  # Minimum and maximum cutoff for C-C bond detection
}

torsion_params = {
    "v_n": 0.3,  # kcal/mol, torsional barrier
    "n": 3,      # Periodicity of torsion angle
}


# Example usage:

# Define atomic coordinates (mock data)


# Define bending energy constants (k_a in kcal/mol/deg^2)
ka_values_degree = {
    ('H', 'C', 'H'): 35,
    ('H', 'C', 'C'): 35,
    ('C', 'C', 'C'): 60
}

# Parameters
bond_distance_threshold = 1.6  # Bonding threshold in angstroms
equilibrium_angle_degrees = 109.5  # Equilibrium bond angle for sp³ carbons

In [5]:


def bond_length(coord1, coord2):
    return np.linalg.norm(np.array(coord2) - np.array(coord1))

def bond_angle(coord1, coord2, coord3):
    vec1 = np.array(coord1) - np.array(coord2)
    vec2 = np.array(coord3) - np.array(coord2)
    cos_theta = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return np.degrees(np.arccos(cos_theta))



def dihedral_angle(coord1, coord2, coord3, coord4):
    # Define the vectors
    vec1 = np.array(coord1) - np.array(coord2)  # r_AB
    vec2 = np.array(coord3) - np.array(coord2)  # r_BC
    vec3 = np.array(coord4) - np.array(coord3)  # r_CD

    # Calculate normal vectors to planes
    t = np.cross(vec1, vec2)  # Normal to plane ABC
    u = np.cross(vec2, vec3)  # Normal to plane BCD

    # Calculate vector perpendicular to both normal vectors
    v = np.cross(t, u)

    # Compute cos(phi)
    cos_phi = np.dot(t, u) / (np.linalg.norm(t) * np.linalg.norm(u))

    # Compute sin(phi)
    sin_phi = np.dot(vec2 / np.linalg.norm(vec2), v) / (np.linalg.norm(t) * np.linalg.norm(u))

    # Correct numerical inaccuracies (clipping for cos_phi)
    cos_phi = np.clip(cos_phi, -1.0, 1.0)

    # Calculate dihedral angle
    angle_rad = np.arctan2(sin_phi, cos_phi)  # Angle in radians
    angle_degree = np.degrees(angle_rad)     # Convert radians to degrees

    # Normalize angle to the range 0° to 360°
    if angle_degree < 0:
        angle_degree += 360
    angle_rad = np.radians(angle_degree)

  #  return angle_degree, angle_rad

    # Subtract the angle from 180 degrees
    corrected_angle_degree = angle_degree -180
    corrected_angle_rad = np.radians(corrected_angle_degree)

    return corrected_angle_degree, corrected_angle_rad


In [6]:
bonds_new = []
def calculate_stretch_energy(atoms, bond_params, bond_cutoffs):
    stretch_energy = 0.0
    visited_bonds = set()  # To avoid double-counting bonds
    bond_debug_info = []  # Store debugging info for later printing

    for i in range(len(atoms) - 1):
        for j in range(i + 1, len(atoms)):
            bond_type = tuple(sorted((atoms[i][0], atoms[j][0])))  # Determine bond type dynamically

            # Only consider bond types defined in bond_params and bond_cutoffs
            if bond_type in bond_params and bond_type in bond_cutoffs:
                # Calculate bond length
                r = bond_length(atoms[i][1:], atoms[j][1:])

                # Apply distance cutoff to avoid counting non-bonded pairs
                min_cutoff, max_cutoff = bond_cutoffs[bond_type]
                if min_cutoff <= r <= max_cutoff and (i, j) not in visited_bonds:
                    visited_bonds.add((i, j))  # Mark this bond as visited

                    # Get the bond parameters
                    k_b, r_0 = bond_params[bond_type]
                    bonds_new.append((i, j))

                    # Calculate energy for this bond and add to total
                    bond_energy = k_b * (r - r_0) ** 2
                    stretch_energy += bond_energy

                    # Store debugging info
                    bond_debug_info.append((i + 1, j + 1, atoms[i][0], atoms[j][0], r, bond_energy))

    # Print bond info
    print("List of all bonds: (Atom1 - Atom2, with labels, and distance in Angstrom, energy contrib in kcal/mol)")
    for bond_info in bond_debug_info:
        print(f"{bond_info[2]:<2} {bond_info[0]:<2} - {bond_info[3]:<2} {bond_info[1]:<2}: {bond_info[4]:>8.5f} {bond_info[5]:>10.5f}")

    return stretch_energy

# Calculate stretching energy
stretch_energy = calculate_stretch_energy(atoms, bond_params, bond_cutoffs)
#print(f"\nTotal Stretching Energy: {stretch_energy:.6f} kcal/mol")
print(f"\nNumber of stretching bonds: {len(bonds_new)}")




List of all bonds: (Atom1 - Atom2, with labels, and distance in Angstrom, energy contrib in kcal/mol)
C  1  - C  2 :  1.51365    0.08017
C  1  - H  3 :  1.06471    0.71806
C  1  - H  4 :  1.07908    0.33453
C  1  - H  5 :  1.22363    4.51897
C  2  - H  6 :  1.05032    1.24660
C  2  - H  7 :  1.10333    0.01555
C  2  - H  8 :  1.13045    0.14631

Number of stretching bonds: 7


In [7]:
angles_new = []
def calculate_bend_energy(atoms, bond_distance_threshold, equilibrium_angle_degrees, ka_values_degree):
    bend_energy_radians = 0.0
    debug_info = []  # Store debugging information

    for j in range(len(atoms)):
        if atoms[j][0] == 'C':  # Central atom must be carbon (customize as needed)
            # Find atoms bonded to the central carbon atom
            bonded_atoms = [
                i for i in range(len(atoms))
                if i != j and bond_length(atoms[i][1:], atoms[j][1:]) <= bond_distance_threshold
            ]

            # Iterate over unique pairs of bonded atoms
            for m in range(len(bonded_atoms)):
                for n in range(m + 1, len(bonded_atoms)):
                    i, k = bonded_atoms[m], bonded_atoms[n]

                    # Determine the angle type and corresponding k_a value
                    angle_type = (atoms[i][0], atoms[j][0], atoms[k][0])
                    ka_degree = ka_values_degree.get(angle_type, 35)  # Default to 35 if not found
                    angles_new.append((i, j, k))  # Store the angle triplet

                    # Calculate bond angle in degrees
                    angle_degrees = bond_angle(atoms[i][1:], atoms[j][1:], atoms[k][1:])

                    # Calculate bending energy in degrees
                    angle_energy_degrees = ka_degree * (angle_degrees - equilibrium_angle_degrees) ** 2

                    # Convert energy from degrees to radians
                    conversion_factor = (np.pi / 180) ** 2
                    angle_energy_radians = angle_energy_degrees * conversion_factor
                    bend_energy_radians += angle_energy_radians

                    # Append debugging information
                    debug_info.append((
                        atoms[i][0], i + 1, atoms[j][0], j + 1, atoms[k][0], k + 1,
                        np.radians(angle_degrees), angle_degrees, angle_energy_radians
                    ))

    # Print bond angle info
    print("List of all bending angles: (At1 - At2 - At3, with labels, angle in radian then degrees, energy contribution)")
    for angle_info in debug_info:
        print(f"{angle_info[0]:<2} {angle_info[1]:<2} - {angle_info[2]:<2} {angle_info[3]:<2} - {angle_info[4]:<2} {angle_info[5]:<2}: {angle_info[6]:>8.5f} {angle_info[7]:>8.3f} {angle_info[8]:>10.5f}")

    return bend_energy_radians



# Calculate bending energy

bend_energy_radians = calculate_bend_energy( atoms, bond_distance_threshold, equilibrium_angle_degrees, ka_values_degree)
print(f"\nNumber of bending angles: {len(angles_new)}")
#print(f"\nTotal Bending Energy in Radians: {bend_energy_radians:.6f} kcal/mol")



List of all bending angles: (At1 - At2 - At3, with labels, angle in radian then degrees, energy contribution)
C  2  - C  1  - H  3 :  1.93391  110.805    0.01815
C  2  - C  1  - H  4 :  1.97631  113.234    0.14868
C  2  - C  1  - H  5 :  1.84668  105.807    0.14540
H  3  - C  1  - H  4 :  1.94949  111.697    0.05148
H  3  - C  1  - H  5 :  1.66416   95.349    2.13495
H  4  - C  1  - H  5 :  2.06830  118.505    0.86448
C  1  - C  2  - H  6 :  1.98810  113.910    0.20732
C  1  - C  2  - H  7 :  1.88713  108.124    0.02018
C  1  - C  2  - H  8 :  1.91190  109.544    0.00002
H  6  - C  2  - H  7 :  1.93566  110.905    0.02106
H  6  - C  2  - H  8 :  1.89944  108.830    0.00479
H  7  - C  2  - H  8 :  1.83539  105.160    0.20081

Number of bending angles: 12


In [8]:
Torsions_bond =[]

def calculate_torsion_energy(atoms, torsion_params):
    """
    Calculate torsion energy for a given set of atoms, torsion parameters, and bond threshold.

    Parameters:
        atoms (list of lists): Atomic coordinates and types, e.g., [['C', x, y, z], ['H', x, y, z], ...].
        torsion_params (dict): Torsion energy parameters {'v_n': value, 'n': periodicity}.
        bond_distance_threshold (float): Threshold for considering atoms as bonded (default 1.6 Å).

    Returns:
        float: Total torsion energy (kcal/mol).
    """
    coords = [atom[1:] for atom in atoms]  # Extract coordinates
    n_atoms = len(atoms)
    torsions = []
    connectivity = [[] for _ in range(n_atoms)]  # Bond connectivity matrix

    # Build connectivity matrix (bond list)
    for i in range(n_atoms):
        for j in range(i + 1, n_atoms):
            if bond_length(coords[i], coords[j]) < bond_distance_threshold:  # Threshold for bonding
                connectivity[i].append(j)
                connectivity[j].append(i)

    # Find all torsions by iterating over atom pairs
    torsion_energy_total_deg = 0.0
    torsion_energy_total_rad = 0.0
    debug_info = []  # Collect debug info for torsions
    for j in range(n_atoms):
        for a in range(len(connectivity[j])):
            k = connectivity[j][a]
            if k < j:
                continue
            for b in range(len(connectivity[j])):
                i = connectivity[j][b]
                if i == k:
                    continue
                for c in range(len(connectivity[k])):
                    l = connectivity[k][c]
                    if l == j or l == i:
                        continue
                    # Calculate dihedral angle
                    angle_deg, angle_rad = dihedral_angle(coords[i], coords[j], coords[k], coords[l])
                    Torsions_bond.append((i, j, k,l))
                    # Calculate torsion energy in degrees and radians
                    torsion_energy_deg = torsion_params["v_n"] * (1 + np.cos(torsion_params["n"] * np.radians(angle_deg)))
                    torsion_energy_rad = torsion_params["v_n"] * (1 + np.cos(torsion_params["n"] * angle_rad))

                    # Append torsion data (atom indices, angles, energies)
                    torsions.append((i, j, k, l, angle_deg, angle_rad, torsion_energy_deg, torsion_energy_rad))
                    torsion_energy_total_deg += torsion_energy_deg
                    torsion_energy_total_rad += torsion_energy_rad

                    # Collect debug info
                    debug_info.append(
                        f"{atoms[i][0]}{i + 1} - {atoms[j][0]}{j + 1} - {atoms[k][0]}{k + 1} - {atoms[l][0]}{l + 1}: "
                        f"Angle (deg): {angle_deg:>10.3f}, Energy (deg): {torsion_energy_deg:>20.6f}, "
                        f"Energy (rad): {torsion_energy_rad:>20.6f}"
                    )

    # Print torsions and energies outside the loop
    print("List of all bending angles: (At1 - At2 - At3, At4, with labels, angle in radian then degrees, energy contribution)")
    for info in debug_info:
        print(info)

    # Print total torsion energy
    print(f"Total torsion energy (calculated using radians): {torsion_energy_total_rad:.6f} kcal/mol")
    torsion_energy_total_rad = float(torsion_energy_total_rad)
    return torsion_energy_total_rad

# Call the function to calculate torsion energy
torsion_energy_total_rad = calculate_torsion_energy(atoms, torsion_params)

# Print the total torsion energy
#print(f"\nTorsion Energy (Radians): {torsion_energy_total_rad:.6f} kcal/mol")
#print(type(torsion_energy_total_rad))



print(f"Number of torsions calculated: {len(Torsions_bond )}")

List of all bending angles: (At1 - At2 - At3, At4, with labels, angle in radian then degrees, energy contribution)
H3 - C1 - C2 - H6: Angle (deg):     57.314, Energy (deg):             0.002961, Energy (rad):             0.002961
H3 - C1 - C2 - H7: Angle (deg):    -66.452, Energy (deg):             0.016959, Energy (rad):             0.016959
H3 - C1 - C2 - H8: Angle (deg):    179.457, Energy (deg):             0.000121, Energy (rad):             0.000121
H4 - C1 - C2 - H6: Angle (deg):   -176.279, Energy (deg):             0.005677, Energy (rad):             0.005677
H4 - C1 - C2 - H7: Angle (deg):     59.955, Energy (deg):             0.000001, Energy (rad):             0.000001
H4 - C1 - C2 - H8: Angle (deg):    -54.136, Energy (deg):             0.014029, Energy (rad):             0.014029
H5 - C1 - C2 - H6: Angle (deg):    -44.879, Energy (deg):             0.089212, Energy (rad):             0.089212
H5 - C1 - C2 - H7: Angle (deg):   -168.646, Energy (deg):             0.051470, 

In [9]:
Total_energy_vanderwaals = 0
Atom_bond_pair =[]
def calculate_and_print_vdw_energy(atoms, bonds_new, angles_new):
    """
    Calculate van der Waals energy for all atom pairs, excluding bonded pairs and angles.
    Print all details after calculation.
    """
    # Define epsilon and sigma values for Hydrogen and Carbon
    epsilon_values = {'H': 0.03, 'C': 0.07}  # in kcal/mol
    sigma_values = {'H': 1.20, 'C': 1.75}    # in Angstroms

    total_energy = 0.0
    debug_info = []  # Store debug information for each pair

    N = len(atoms)

    # Iterate over all unique atom pairs
    for i in range(N):
        for j in range(i + 1, N):
            calc_vdw = True  # Assume we will calculate vdW for this pair
            # Exclude bonded pairs
            for bond in bonds_new:
                if i in bond and j in bond:
                    calc_vdw = False
                    break
            # Exclude pairs involved in angles
            if calc_vdw:
                for angle in angles_new:
                    if i in angle and j in angle:
                        calc_vdw = False
                        break

            # Retrieve atom types and coordinates
            atom1_type, x1, y1, z1 = atoms[i]
            atom2_type, x2, y2, z2 = atoms[j]
            r_ij = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

            if calc_vdw:
                # Retrieve epsilon and sigma values
                epsilon_i = epsilon_values.get(atom1_type)
                epsilon_j = epsilon_values.get(atom2_type)
                sigma_i = sigma_values.get(atom1_type)
                sigma_j = sigma_values.get(atom2_type)

                if epsilon_i is None or epsilon_j is None:
                    raise ValueError(f"Unknown atom type: {atom1_type} or {atom2_type}")

                # Compute mixed epsilon and sigma using geometric mean
                epsilon_ij = math.sqrt(epsilon_i * epsilon_j)
                sigma_ij = 2 * math.sqrt(sigma_i * sigma_j)

                # Compute Lennard-Jones potential
                term12 = (sigma_ij / r_ij) ** 12
                term6 = (sigma_ij / r_ij) ** 6
                energy_lj = 4 * epsilon_ij * (term12 - term6)

                # Accumulate the total energy
                total_energy += energy_lj
                energy = energy_lj
            else:
                # For uncalculated pairs, energy is zero
                energy = 0.0

            # Store the current pair details in debug info
            debug_info.append(
                f"{atom1_type} {i + 1:<3}- {atom2_type} {j + 1:<3}: {r_ij:>15.4f} {energy:>20.4f}"
            )
            Atom_bond_pair.append((atom1_type, i + 1, atom2_type, j + 1))
    total_energy = float(total_energy)
    #Total_energy_vanderwaals.append(total_energy)

    # Print debug information and total energy after the loop
    print(f"{'Atom Pair':<15} {'Distance (Å)':>15} {'Energy (kcal/mol)':>20}")
    for info in debug_info:
        print(info)
    print(f"\nTotal van der Waals Energy: {total_energy:.4f} kcal/mol")

    return total_energy


# Example Input
# Calculate and print van der Waals energy
van_der_waal_ener = calculate_and_print_vdw_energy(atoms, bonds_new, angles_new)

#print(bonds_new)
#print(type(van_der_waal_ener))
#print(van_der_waal_ener)
#print(type(van_der_waal_ener))




print(f"Number of van der waal calculated: {len(Atom_bond_pair )}")

Atom Pair          Distance (Å)    Energy (kcal/mol)
C 1  - C 2  :          1.5137               0.0000
C 1  - H 3  :          1.0647               0.0000
C 1  - H 4  :          1.0791               0.0000
C 1  - H 5  :          1.2236               0.0000
C 1  - H 6  :          2.1640               0.0000
C 1  - H 7  :          2.1325               0.0000
C 1  - H 8  :          2.1711               0.0000
C 2  - H 3  :          2.1377               0.0000
C 2  - H 4  :          2.1781               0.0000
C 2  - H 5  :          2.1903               0.0000
C 2  - H 6  :          1.0503               0.0000
C 2  - H 7  :          1.1033               0.0000
C 2  - H 8  :          1.1304               0.0000
H 3  - H 4  :          1.7741               0.0000
H 3  - H 5  :          1.6952               0.0000
H 3  - H 6  :          2.5002              -0.0204
H 3  - H 7  :          2.5003              -0.0204
H 3  - H 8  :          3.0658              -0.0213
H 4  - H 5  :          1.9804

In [19]:
internal_coords = len(angles_new) + len(bonds_new) + len(Torsions_bond )
cartesian_coords = 3 * len(atoms)
print(f"Number of internal coordinates: {internal_coords}")
print(f"Number of cartesian coordinates: {cartesian_coords}")



Number of internal coordinates: 293
Number of cartesian coordinates: 24
Number of coordinates:
Streching: 126 Bending: 77 Torsion: 90 Van der waal: 308
internal coordinates: 293 cartesian coordinates: 24


The input file has: 8 atoms
Atoms and coordinates (in Å):
C   -0.7560   0.0500   0.0000
C    0.7560   0.0000   0.0500
H   -1.1404   0.6586   0.7845
H   -1.1404   0.3501  -0.9626
H   -1.1405  -1.0087   0.4781
H    1.1404  -0.3501   0.9626
H    1.1405   1.0087  -0.1781
H    1.1404  -0.6586  -0.7845
List of all bonds: (Atom1 - Atom2, with labels, and distance in Angstrom, energy contrib in kcal/mol)
C  1  - C  2 :  1.51365    0.08017
C  1  - H  3 :  1.06471    0.71806
C  1  - H  4 :  1.07908    0.33453
C  1  - H  5 :  1.22363    4.51897
C  2  - H  6 :  1.05032    1.24660
C  2  - H  7 :  1.10333    0.01555
C  2  - H  8 :  1.13045    0.14631
List of all bending angles: (At1 - At2 - At3, with labels, angle in radian then degrees, energy contribution)
C  2  - C  1  - H  3 :  1.93391  110.805    0.01815
C  2  - C  1  - H  4 :  1.97631  113.234    0.14868
C  2  - C  1  - H  5 :  1.84668  105.807    0.14540
H  3  - C  1  - H  4 :  1.94949  111.697    0.05148
H  3  - C  1  - H  5 :  1.66416   95.

'\nstretching_gradients = calculate_stretching_gradient(atoms, bond_params, bond_cutoffs)\nbending_gradients = bending_energy_with_gradients(atoms, ka_values_degree, bond_distance_threshold)\ntorsion_gradients = calculate_and_print_torsions_with_gradient(atoms, torsion_params)\nvan_der_waal_gradients = calculate_and_print_vdw_energy_with_gradient(atoms, bonds_new, angles_new)\nAnalytical_gradient_of_overall_energy = total_gradient, energy_data = calculate_gradient_for_structure(atoms)\noptimization_cycle(atoms, M_k, max_cycles=1)\n'

In [12]:
import os
import sys

def calculate_total_energy(atoms):
    """
    Calculate the total energy by integrating individual energy components:
    van der Waals, torsion, bending, and stretching energies.
    """
    # Redirect output to suppress intermediate prints
    with open(os.devnull, 'w') as fnull:
        old_stdout = sys.stdout
        sys.stdout = fnull
        try:
            # Calculate van der Waals energy
            van_der_waal_ener = calculate_and_print_vdw_energy(atoms, bonds_new, angles_new)
            van_der_waal_ener = float(van_der_waal_ener)  # Ensure scalar

            # Calculate torsion energy
            torsion_energy = calculate_torsion_energy(atoms, torsion_params)
            torsion_energy = float(torsion_energy)  # Ensure scalar

            # Calculate bending energy
            bend_energy_radians = calculate_bend_energy(atoms, bond_distance_threshold, equilibrium_angle_degrees, ka_values_degree)
            bend_energy_radians = float(bend_energy_radians)  # Ensure scalar

            # Calculate stretching energy
            stretch_energy = calculate_stretch_energy(atoms, bond_params, bond_cutoffs)
            stretch_energy = float(stretch_energy)  # Ensure scalar
        finally:
            # Restore original output
            sys.stdout = old_stdout

    # Total energy
    total_energy = van_der_waal_ener + torsion_energy + bend_energy_radians + stretch_energy

    return float(total_energy)

# Example usage
total_energy = calculate_total_energy(atoms)
print(f"\nTotal Energy: {total_energy:.6f} kcal/mol")
print(type(total_energy))



Total Energy: 10.992616 kcal/mol
<class 'float'>


In [13]:
import numpy as np

def bond_length(coord1, coord2):
    """Calculate bond length between two atoms."""
    return np.linalg.norm(np.array(coord1) - np.array(coord2))

def calculate_stretching_gradient(atoms, bond_params, bond_cutoffs):
    """
    Calculate bond stretching energy and its gradients.

    Parameters:
    - atoms: List of atoms with their coordinates
    - bond_params: Dictionary with bond parameters (k_b, r_0)
    - bond_cutoffs: Dictionary with bond cutoff distances

    Returns:
    - stretch_energy: Total bond stretching energy
    - stretching_gradients: Gradients of bond stretching energy for all atoms
    """
    coords = [np.array(atom[1:]) for atom in atoms]  # Ensure coordinates are numpy arrays
    n_atoms = len(atoms)
    visited_bonds = set()
    stretching_gradients = np.zeros((3, n_atoms))
    stretch_energy = 0.0
    bond_data = []

    for i in range(n_atoms - 1):
        for j in range(i + 1, n_atoms):
            bond_type = tuple(sorted((atoms[i][0], atoms[j][0])))

            if bond_type in bond_params and bond_type in bond_cutoffs:
                r = bond_length(coords[i], coords[j])
                min_cutoff, max_cutoff = bond_cutoffs[bond_type]

                if min_cutoff <= r <= max_cutoff and (i, j) not in visited_bonds:
                    visited_bonds.add((i, j))

                    k_b, r_0 = bond_params[bond_type]
                    delta_r = r - r_0

                    # Stretching energy
                    bond_energy = k_b * delta_r ** 2
                    stretch_energy += bond_energy

                    # Gradient calculation using the formula g_stretch(r) = 2 * k_b * delta_r * d(r)/dX
                    grad_factor = 2 * k_b * delta_r / r
                    grad_i = grad_factor * (coords[i] - coords[j])
                    grad_j = -grad_i

                    # Accumulate gradients
                    stretching_gradients[:, i] += grad_i
                    stretching_gradients[:, j] += grad_j

                    # Store bond information for printing
                    bond_data.append((atoms[i][0], i + 1, atoms[j][0], j + 1, r, bond_energy))

    # Print results
    #print("\nList of all bond lengths with energies:")
   # for atom1, idx1, atom2, idx2, length, energy in bond_data:
       # print(f"{atom1}{idx1} - {atom2}{idx2}: {length:>10.3f} {energy:>10.6f}")

    #print(f"\nTotal Bond Stretching Energy: {stretch_energy:.6f} kcal/mol")
    #print("Number of bonds calculated:", len(bond_data))

    print("\nAnalytical gradient of stretching energy:")
    for atom_index, (atom, grad) in enumerate(zip(atoms, stretching_gradients.T)):
        print(f"{atom[0]}{atom_index + 1:>2} {grad[0]:>10.6f} {grad[1]:>10.6f} {grad[2]:>10.6f}")

    return stretching_gradients

stretching_gradients = calculate_stretching_gradient(atoms, bond_params, bond_cutoffs)



Analytical gradient of stretching energy:
C 1  15.635064  92.637060 -26.697229
C 2   2.251023  -0.996772  45.574735
H 3  11.447179 -18.123708 -23.361894
H 4   7.709220  -6.018567  19.305139
H 5 -24.993693 -68.818786  31.077984
H 6 -15.289332  13.925065 -36.298241
H 7  -1.626161  -4.266082   0.964700
H 8   4.866699  -8.338211 -10.565194


In [14]:


def bending_energy_with_gradients(atoms, ka_values_degree, bond_distance_threshold):
    """
    Calculate bending energy and its gradients using the cross product method.

    Parameters:
    - atoms: List of atoms with coordinates
    - ka_values_degree: Dictionary with force constants for angle types
    - bond_distance_threshold: Threshold to determine bonded atoms
    """
    equilibrium_angle_radians = np.radians(109.5)  # Default equilibrium angle for sp3 hybridization
    bending_gradients = np.zeros((len(atoms), 3))  # Gradient array
    total_bending_energy = 0.0

    for j in range(len(atoms)):  # Loop through central atoms
        if atoms[j][0] == 'C':  # Focus on carbon atoms as central atoms
            bonded_atoms = [
                i for i in range(len(atoms))
                if i != j and bond_length(atoms[i][1:], atoms[j][1:]) <= bond_distance_threshold
            ]

            for m in range(len(bonded_atoms)):
                for n in range(m + 1, len(bonded_atoms)):
                    i, k = bonded_atoms[m], bonded_atoms[n]
                    angle_type = (atoms[i][0], atoms[j][0], atoms[k][0])
                    ka_degree = ka_values_degree.get(angle_type, 35)

                    # Compute vectors
                    r_BA = np.array(atoms[j][1:]) - np.array(atoms[i][1:])
                    r_BC = np.array(atoms[j][1:]) - np.array(atoms[k][1:])
                    p = np.cross(r_BA, r_BC)  # Cross product of the vectors

                    r_AB = np.linalg.norm(r_BA)
                    r_BC_norm = np.linalg.norm(r_BC)
                    norm_p = np.linalg.norm(p)

                    cos_theta = np.dot(r_BA, r_BC) / (r_AB * r_BC_norm)
                    theta = np.arccos(np.clip(cos_theta, -1.0, 1.0))
                    delta_theta = theta - equilibrium_angle_radians

                    # Bending energy
                    energy = ka_degree * delta_theta**2
                    total_bending_energy += energy

                    # Gradient calculation
                    grad_A = -np.cross(r_BA, p) / (r_AB**2 * norm_p) * 2 * ka_degree * delta_theta
                    grad_C = np.cross(r_BC, p) / (r_BC_norm**2 * norm_p) * 2 * ka_degree * delta_theta
                    grad_B = -(grad_A + grad_C)

                    # Accumulate gradients
                    bending_gradients[i] += grad_A
                    bending_gradients[j] += grad_B
                    bending_gradients[k] += grad_C

    # Print results
   # print(f"Total Bending Energy: {total_bending_energy:.6f} kcal/mol")
    print("Analytical Gradient of Bending Energy:")
    for atom_index, grad in enumerate(bending_gradients):
        print(f"{atoms[atom_index][0]}{atom_index + 1:>2} {grad[0]:>10.6f} {grad[1]:>10.6f} {grad[2]:>10.6f}")

    return bending_gradients





bending_gradients = bending_energy_with_gradients(atoms, ka_values_degree, bond_distance_threshold)


Analytical Gradient of Bending Energy:
C 1   0.055103   7.742266 -35.045561
C 2  -7.191111  -7.290485  16.053765
H 3  -5.739392 -14.842073   8.701929
H 4   2.987536   6.339252   0.783296
H 5   3.139544   7.316357  18.726170
H 6   4.311326  -1.387258  -2.348184
H 7  -0.097287  -1.189726  -5.425178
H 8   2.534282   3.311665  -1.446237


In [15]:

def calculate_and_print_torsions_with_gradient(atoms, torsion_params):
    """
    Calculate torsion angles, torsion energies, and their gradients.

    Parameters:
    - atoms: List of atoms with their coordinates
    - torsion_params: Dictionary with torsion parameters (v_n, n)

    Returns:
    - torsion_energy_total_rad: Total torsion energy in radians
    - dihedrals: List of dihedrals (atom indices)
    - torsion_gradients: Gradients of torsion energy for all atoms
    """
    coords = [np.array(atom[1:]) for atom in atoms]  # Ensure coordinates are numpy arrays
    n_atoms = len(atoms)
    torsions = []
    dihedrals = []  # Store dihedrals for Wilson B matrix
    connectivity = [[] for _ in range(n_atoms)]  # Bond connectivity matrix
    torsion_gradients = np.zeros((3, n_atoms))  # Gradient array

    # Build connectivity matrix (bond list)
    for i in range(n_atoms):
        for j in range(i + 1, n_atoms):
            if bond_length(coords[i], coords[j]) < 1.6:  # Threshold for bonding
                connectivity[i].append(j)
                connectivity[j].append(i)

    # Find all torsions and calculate energies
    for j in range(n_atoms):
        for a in range(len(connectivity[j])):
            k = connectivity[j][a]
            if k < j:
                continue
            for b in range(len(connectivity[j])):
                i = connectivity[j][b]
                if i == k:
                    continue
                for c in range(len(connectivity[k])):
                    l = connectivity[k][c]
                    if l == j or l == i:
                        continue

                    # Calculate dihedral angle
                    angle_deg, angle_rad = dihedral_angle(coords[i], coords[j], coords[k], coords[l])

                    # Save dihedral for Wilson B matrix
                    dihedrals.append((i, j, k, l))

                    # Calculate torsion energy
                    torsion_energy_rad = torsion_params["v_n"] * (1 + np.cos(torsion_params["n"] * angle_rad))
                    torsions.append((i, j, k, l, angle_deg, angle_rad, torsion_energy_rad))

                    # Gradient calculation
                    grad_term = -torsion_params['n'] * torsion_params['v_n'] * np.sin(torsion_params['n'] * angle_rad)
                    b1 = coords[j] - coords[i]
                    b2 = coords[k] - coords[j]
                    b3 = coords[l] - coords[k]

                    t = np.cross(b1, b2)
                    u = np.cross(b2, b3)
                    norm_t = np.linalg.norm(t)
                    norm_u = np.linalg.norm(u)

                    term1 = np.cross(t, b2) / (norm_t**2 * np.linalg.norm(b2))
                    term2 = np.cross(-u, b2) / (norm_u**2 * np.linalg.norm(b2))

                    # Gradients for each atom
                    grad_i = grad_term * np.cross(term1, b2)
                    grad_j = grad_term * (np.cross(coords[k] - coords[i], term1) + np.cross(term2, b3))
                    grad_k = grad_term * (np.cross(term1, b1) + np.cross(coords[l] - coords[j], term2))
                    grad_l = grad_term * np.cross(term2, b2)

                    # Accumulate gradients
                    torsion_gradients[:, i] += grad_i
                    torsion_gradients[:, j] += grad_j
                    torsion_gradients[:, k] += grad_k
                    torsion_gradients[:, l] += grad_l

    # Print torsions and gradients
    torsion_energy_total_rad = 0.0
    #print("\nList of all torsion angles with energies:")
    #for (i, j, k, l, angle_deg, angle_rad, torsion_energy_rad) in torsions:
     #   torsion_energy_total_rad += torsion_energy_rad
      #  print(f"{atoms[i][0]}{i + 1} - {atoms[j][0]}{j + 1} - {atoms[k][0]}{k + 1} - {atoms[l][0]}: "
       #       f"{angle_deg:>10.3f} {angle_rad:>10.3f} {torsion_energy_rad:>20.6f}")

    #print(f"\nTotal torsion energy (calculated using radians): {torsion_energy_total_rad:.6f} kcal/mol")
    #print(f"Number of torsions calculated: {len(torsions)}")

    print("\nAnalytical gradient of torsional energy:")
    for atom_index, (atom, grad) in enumerate(zip(atoms, torsion_gradients.T)):
        print(f"{atom[0]}{atom_index + 1:>2} {grad[0]:>10.6f} {grad[1]:>10.6f} {grad[2]:>10.6f}")

    return torsion_gradients
torsion_gradients = calculate_and_print_torsions_with_gradient(atoms, torsion_params)




Analytical gradient of torsional energy:
C 1   0.075425   0.201184  -2.079683
C 2  -0.041803  -0.149998   1.114111
H 3  -0.020908  -0.361807   0.270439
H 4  -0.009885  -0.428613  -0.129677
H 5  -0.025619   0.654916   1.429636
H 6  -0.013876  -0.671302  -0.251686
H 7  -0.004775   0.044550   0.188958
H 8   0.041441   0.711070  -0.542098


In [16]:

# Total van der Waals energy container
Total_energy_vanderwaals = []

def calculate_and_print_vdw_energy_with_gradient(atoms, bonds_new, angles_new):
    # Define epsilon and sigma values for Hydrogen and Carbon
    epsilon_values = {'H': 0.03, 'C': 0.07}  # in kcal/mol
    sigma_values = {'H': 1.20, 'C': 1.75}    # in Angstroms

    total_energy = 0.0
    gradients = np.zeros((len(atoms), 3))  # Initialize gradients
    van_der_waal_gradients = []
    #print(f"{'Atom Pair':<15} {'Distance (Å)':>15} {'Energy (kcal/mol)':>20}")

    N = len(atoms)

    # Iterate over all unique atom pairs
    for i in range(N):
        for j in range(i + 1, N):
            calc_vdw = True  # Assume we will calculate vdW for this pair
            # Exclude bonded pairs
            for bond in bonds_new:
                if i in bond and j in bond:
                    calc_vdw = False
                    break
            # Exclude pairs involved in angles
            if calc_vdw:
                for angle in angles_new:
                    if i in angle and j in angle:
                        calc_vdw = False
                        break

            # Retrieve atom types and coordinates
            atom1_type, x1, y1, z1 = atoms[i]
            atom2_type, x2, y2, z2 = atoms[j]
            r_ij_vector = np.array([x1 - x2, y1 - y2, z1 - z2])
            r_ij = np.linalg.norm(r_ij_vector)

            if calc_vdw:
                # Retrieve epsilon and sigma values
                epsilon_i = epsilon_values.get(atom1_type)
                epsilon_j = epsilon_values.get(atom2_type)
                sigma_i = sigma_values.get(atom1_type)
                sigma_j = sigma_values.get(atom2_type)

                if epsilon_i is None or epsilon_j is None:
                    raise ValueError(f"Unknown atom type: {atom1_type} or {atom2_type}")

                # Compute mixed epsilon and sigma using geometric mean
                epsilon_ij = math.sqrt(epsilon_i * epsilon_j)
                sigma_ij = 2 * math.sqrt(sigma_i * sigma_j)

                # Compute Lennard-Jones potential energy
                term12 = (sigma_ij / r_ij) ** 12
                term6 = (sigma_ij / r_ij) ** 6
                energy_lj = 4 * epsilon_ij * (term12 - term6)
                total_energy += energy_lj

                # Compute Lennard-Jones gradient
                A_ij = 4 * epsilon_ij * sigma_ij**12
                B_ij = 4 * epsilon_ij * sigma_ij**6
                grad_prefactor = (-12 * A_ij / r_ij**14 + 6 * B_ij / r_ij**8)
                gradient = grad_prefactor * r_ij_vector

                # Accumulate gradients
                gradients[i] += gradient
                gradients[j] -= gradient  # Opposite direction for atom j

                # Print the current pair details
             #   print(f"{atom1_type} {i + 1:<3}- {atom2_type} {j + 1:<3}: {r_ij:>15.4f} {energy_lj:>20.4f}")
            #else:

          #    energy_lj = 0.0

   # Total_energy_vanderwaals.append(total_energy)
    van_der_waal_gradients.append(gradients)
    # Print total energy and gradients
    #print(f"\nTotal van der Waals Energy: {total_energy:.4f} kcal/mol")
    print("\nAnalytical Gradients of van der waals (kcal/mol/Å):")
    for idx, (atom, grad) in enumerate(zip(atoms, gradients)):
        print(f"{atom[0]}{idx + 1:<2} : {grad[0]:>10.6f} {grad[1]:>10.6f} {grad[2]:>10.6f}")

    return van_der_waal_gradients



# Calculate and print van der Waals energy and gradients
van_der_waal_gradients = calculate_and_print_vdw_energy_with_gradient(atoms, bonds_new, angles_new)



Analytical Gradients of van der waals (kcal/mol/Å):
C1  :   0.000000   0.000000   0.000000
C2  :   0.000000   0.000000   0.000000
H3  :   0.210252  -0.020984  -0.024936
H4  :   0.210246  -0.011177   0.030612
H5  :   0.235217   0.052803   0.044133
H6  :  -0.329073  -0.023134  -0.040606
H7  :  -0.211264  -0.033055   0.003130
H8  :  -0.115377   0.035547  -0.012334


In [17]:

# Helper function to ensure the input is a numpy array
def ensure_numpy_array(array):
    """Convert input to a numpy array if it isn't already."""
    if not isinstance(array, np.ndarray):
        array = np.array(array)
    return array

# Helper function to reshape or transpose arrays to match a target shape
def reshape_to_match(array, target_shape):
    """
    Reshape or transpose an array to match the target shape.
    Handles cases like 1-dimensional expansion or axis swaps.
    """
    array = ensure_numpy_array(array)  # Ensure input is a numpy array
    if array.shape == target_shape:
        return array
    if len(array.shape) == 3 and array.shape[0] == 1:
        array = array[0]  # Remove the first dimension
    if array.T.shape == target_shape:
        return array.T
    try:
        array = array.reshape(target_shape)
    except ValueError:
        raise ValueError(f"Cannot reshape or transpose array of shape {array.shape} to match {target_shape}")
    return array


# Function to calculate the total gradient energy by summing input gradients

def calculate_total_gradient(*gradients):
    """
    Calculate the total gradient energy by summing all input gradients.
    Gradients are reshaped dynamically to match the target shape.
    """
    if not gradients:
        raise ValueError("No gradients provided for calculation.")

    # Ensure all gradients are numpy arrays
    gradients = [ensure_numpy_array(g) for g in gradients]

    # Determine the target shape dynamically based on the first gradient
    target_shape = gradients[0].shape

    # Reshape all gradients to match the target shape
    reshaped_gradients = []
    for gradient in gradients:
        try:
            reshaped_gradients.append(reshape_to_match(gradient, target_shape))
        except ValueError as e:
            raise ValueError(f"Error reshaping gradient with shape {gradient.shape} to {target_shape}: {e}")

    # Sum the reshaped gradients
    return sum(reshaped_gradients)




def calculate_gradient_for_structure(atoms):
    """
    Calculate the total gradient energy for a given structure.
    """
    # Suppress individual gradient energy outputs
    with open(os.devnull, 'w') as fnull:
        with contextlib.redirect_stdout(fnull):
            try:
                # Calculate individual gradients
                stretching_gradients = calculate_stretching_gradient(atoms, bond_params, bond_cutoffs)
                bending_gradient = bending_energy_with_gradients(atoms, ka_values_degree, bond_distance_threshold)
                torsion_energy = calculate_and_print_torsions_with_gradient(atoms, torsion_params)
                van_der_waal_gradients = calculate_and_print_vdw_energy_with_gradient(atoms, bonds_new, angles_new)

            except Exception as e:
                raise e

    # Define the total gradient energy
    try:
        Total_gradient_energy = calculate_total_gradient(
            stretching_gradients,
            bending_gradient,
            torsion_energy,
            van_der_waal_gradients
        )

        # Transpose the result for better readability
        Total_gradient_energy_transposed = Total_gradient_energy.T.tolist()

        # Store energy data for later use
        energy_data = []
        for idx, (atom, energy_row) in enumerate(zip(atoms, Total_gradient_energy_transposed)):
            energy_data.append({
                'atom': atom[0],
                'index': idx + 1,
                'energy': energy_row
            })

        # Print the total analytical gradient in the desired format
        print("Analytical gradient of overall energy: (in kcal/mol/\u00c5)")
        for data in energy_data:
            atom = data['atom']
            energy = data['energy']
            print(f"{atom:2} {energy[0]:10.6f} {energy[1]:10.6f} {energy[2]:10.6f}")

    except ValueError as e:
        print("Error calculating total gradient energy:", e)
        return None, []

    return Total_gradient_energy, energy_data

# Example call to the function after calculating the gradient

Analytical_gradient_of_overall_energy = total_gradient, energy_data = calculate_gradient_for_structure(atoms)

Analytical gradient of overall energy: (in kcal/mol/Å)
C   15.765592 100.580510 -63.822473
C   -4.981891  -8.437254  62.742611
H    5.897132 -33.348572 -14.414463
H   10.897117  -0.119104  19.989371
H  -21.644552 -60.794709  51.277924
H  -11.320955  11.843371 -38.938717
H   -1.939487  -5.444313  -4.268389
H    7.327045  -4.279929 -12.565863


In [26]:

# Function to update the inverse Hessian matrix
def update_inverse_hessian(M_k, y_k, s_k):
    sk_yk_dot = np.dot(s_k, y_k)
    if sk_yk_dot <= 1e-8:
        raise ValueError("Denominator (s_k · y_k) is too small or zero, cannot update Hessian.")

    v_k = M_k @ y_k
    numerator = (np.dot(s_k, y_k) + np.dot(y_k, v_k)) * s_k
    Denominator = sk_yk_dot
    M_k_update = M_k + (np.outer(numerator, s_k) / Denominator**2) - (np.outer(s_k, v_k) + np.outer(v_k, s_k)) / Denominator
    return M_k_update

# Mock atoms data


# Initialize variables
num_atoms = len(atoms)
n = 3 * num_atoms
M_k = np.eye(n) / 300
total_gradient, energy_data = calculate_gradient_for_structure(atoms)
gradient_flat = np.array([entry['energy'] for entry in energy_data]).flatten()
p_k = -np.dot(M_k, gradient_flat)
p_k_reshaped = p_k.reshape(len(atoms), 3)
total_energy = calculate_total_energy(atoms)

def optimization_cycle(atoms, M_k, max_cycles=20):
    gradient, energy_data = calculate_gradient_for_structure(atoms)
    gradient_flat = np.array([entry['energy'] for entry in energy_data]).flatten()

    for cycle in range(max_cycles):
        print(f"\n***** Optimization Cycle {cycle + 1} *****")

        # Compute search direction
        p_k = -np.dot(M_k, gradient_flat)
        p_k_reshaped = p_k.reshape(len(atoms), 3)

        # Perform line search
        initial_alpha = 0.8
        decrement_factor = 0.8
        num_steps = 6
        c1 = 0.1
        total_energy = calculate_total_energy(atoms)

        best_alpha = initial_alpha
        best_energy = float('inf')
        best_coordinates = atoms

        print("Predicted structure change p_k = -M_k grad V(r_k):")
        for idx, (atom, p_row) in enumerate(zip(atoms, p_k_reshaped)):
            print(f"{atom[0]:<2} {p_row[0]:>10.6f} {p_row[1]:>10.6f} {p_row[2]:>10.6f}")

        for step in range(num_steps):
            alpha = initial_alpha * (decrement_factor ** step)
            updated_atoms = [
                [atom[0], atom[1] + alpha * pk[0], atom[2] + alpha * pk[1], atom[3] + alpha * pk[2]]
                for atom, pk in zip(atoms, p_k_reshaped)
            ]
            energy = calculate_total_energy(updated_atoms)
            print(f"Line search: alpha = {alpha:.6f}, energy = {energy:.8f}")
            if energy < best_energy:
                best_alpha = alpha
                best_energy = energy
                best_coordinates = updated_atoms
            if energy <= total_energy + c1 * alpha * np.dot(gradient_flat, p_k):
                break

        # Update gradient and Hessian
        new_gradient, energy_data_new = calculate_gradient_for_structure(best_coordinates)
        new_gradient_flattened = np.array([entry['energy'] for entry in energy_data_new]).flatten()
        y_k = new_gradient_flattened - gradient_flat
        s_k = p_k * best_alpha
        M_k = update_inverse_hessian(M_k, y_k, s_k)

        # Stopping criteria
        gradient_norm = np.linalg.norm(gradient_flat)
        if gradient_norm <= 1e-6:
            print(f"Converged: Gradient norm {gradient_norm:.6e} below tolerance.")
            break

        step_size = np.linalg.norm(s_k)
        if step_size <= 1e-6:
            print(f"Converged: Step size {step_size:.6e} below tolerance.")
            break

        if abs(best_energy - total_energy) <= 1e-6:
            print(f"Converged: Function value change {abs(best_energy - total_energy):.6e} below tolerance.")
            break

        # Update atoms and gradient
        atoms = best_coordinates
        gradient_flat = new_gradient_flattened

        # Print results for this cycle
        print("\nUpdated Inverse Hessian (M_k):")
        for row in M_k:
            print(" ".join(f"{val:10.6f}" for val in row))

        print("\nNew Gradient at Best Coordinates:")
        for entry in energy_data_new:
            print(f"{entry['atom']}{entry['index']:<2}: {entry['energy'][0]:>10.8f} {entry['energy'][1]:>10.6f} {entry['energy'][2]:>10.6f}")

        print("\nFinal Coordinates After Hessian Update:")
        for atom in best_coordinates:
            print(f"{atom[0]:<2} {atom[1]:>10.6f} {atom[2]:>10.6f} {atom[3]:>10.6f}")

    # Print final results
    print("\n###########################")
    print("# Optimization converged #")
    print("###########################")
    print(f"\nFinal energy at minimum: {best_energy:>10.8f} kcal/mol")
    print("Final coordinates:")
    for atom in best_coordinates:
        print(f"{atom[0]:<2} {atom[1]:>10.6f} {atom[2]:>10.6f} {atom[3]:>10.6f}")


optimization_cycle(atoms, M_k, max_cycles=100)


Analytical gradient of overall energy: (in kcal/mol/Å)
C   15.765592 100.580510 -63.822473
C   -4.981891  -8.437254  62.742611
H    5.897132 -33.348572 -14.414463
H   10.897117  -0.119104  19.989371
H  -21.644552 -60.794709  51.277924
H  -11.320955  11.843371 -38.938717
H   -1.939487  -5.444313  -4.268389
H    7.327045  -4.279929 -12.565863
Analytical gradient of overall energy: (in kcal/mol/Å)
C   15.765592 100.580510 -63.822473
C   -4.981891  -8.437254  62.742611
H    5.897132 -33.348572 -14.414463
H   10.897117  -0.119104  19.989371
H  -21.644552 -60.794709  51.277924
H  -11.320955  11.843371 -38.938717
H   -1.939487  -5.444313  -4.268389
H    7.327045  -4.279929 -12.565863

***** Optimization Cycle 1 *****
Predicted structure change p_k = -M_k grad V(r_k):
C   -0.052552  -0.335268   0.212742
C    0.016606   0.028124  -0.209142
H   -0.019657   0.111162   0.048048
H   -0.036324   0.000397  -0.066631
H    0.072149   0.202649  -0.170926
H    0.037737  -0.039478   0.129796
H    0.006465

In [18]:
"""
print_atom_coordinates(atoms)
stretch_energy = calculate_stretch_energy(atoms, bond_params, bond_cutoffs)
bend_energy_radians = calculate_bend_energy(atoms, bond_distance_threshold, equilibrium_angle_degrees, ka_values_degree)
torsion_energy_total_rad = calculate_torsion_energy(atoms, torsion_params)
van_der_waal_ener = calculate_and_print_vdw_energy(atoms, bonds_new, angles_new)

print("Number of coordinates:")
print(f"Streching: {len(angles_new)} Bending: {len(bonds_new)} Torsion: {len(Torsions_bond )} Van der waal: {len(Atom_bond_pair)}")
print(f"internal coordinates: {internal_coords} cartesian coordinates: {cartesian_coords}")

print(f"Number of internal coordinates: {internal_coords}")
print(f"Number of cartesian coordinates: {cartesian_coords}")


#print(f"\nNumber of bending angles: {len(angles_new)}")
#print(f"\nNumber of stretching bonds: {len(bonds_new)}")
#print(f"Number of torsions calculated: {len(Torsions_bond )}")
#print(f"Number of van der waal calculated: {len(Atom_bond_pair )}")

print(f"\nTotal Stretching Energy: {stretch_energy:.6f} kcal/mol")
print(f"\nTotal Bending Energy in Radians: {bend_energy_radians:.6f} kcal/mol")
print(f"\nTorsion Energy (Radians): {torsion_energy_total_rad:.6f} kcal/mol")
print(f"\nTorsion Energy (Radians): {torsion_energy_total_rad:.6f} kcal/mol")
print(f"\nTotal van der Waals Energy: {van_der_waal_ener:.6f} kcal/mol")
#print(f"\nTotal Energy: {total_energy:.6f} kcal/mol")


stretching_gradients = calculate_stretching_gradient(atoms, bond_params, bond_cutoffs)
bending_gradients = bending_energy_with_gradients(atoms, ka_values_degree, bond_distance_threshold)
torsion_gradients = calculate_and_print_torsions_with_gradient(atoms, torsion_params)
van_der_waal_gradients = calculate_and_print_vdw_energy_with_gradient(atoms, bonds_new, angles_new)
Analytical_gradient_of_overall_energy = total_gradient, energy_data = calculate_gradient_for_structure(atoms)
optimization_cycle(atoms, M_k, max_cycles=1)
"""